# First Function Description

1. This function takes 2 inputs.

2. First is a small dataframe for a specific user, second is the test point in which we have to give predictions

3. Since we have all the columns in given dataframe. So we create multiple new dataframe each with their unique features based on column values of given small dataframe.

4. Later on in the function we merge all this dataframe created. The given dataframe has reordered column too which is our true label so we get reordered features too. 

5. Our test_point do not have the reordered column as we have to predict on it.

6. We merge our given dataframe few columns with the merged features dataframe created.

7. Now we drop the 'reordered' and merge this dataframe with the test point. So now we have all the features in test point too.

8. Now we load our best model and finally predict the dataframe as well as test point.

9. Now we add the results as column in dataframe and we create a dictionary with this dataframe such that we have keys as the order_id and values is the list of products for the specific order_id and user_id. This is our final prediction.

#There maybe several question 

<b>[Q].</b> Why we need dataframe and test point as input in function and not just the test point?

<b>[A].</b> 1.As per the problem definition we have to predict the products which can be reordered by user. And if we get a new user there is no point of predicting products to be reordered. As user never ordered before.

2. The dataframe contains previous order history of new user. Also it has reorderd column which is the label here. So with this dataframe we are able to build the features and then we merge this features dataframe with the final test point and we get prediction.

<b>[Q].</b> Why we have the code for dictionary. And why do we return values of this dictionary?

<b>[A].</b> The problem we have is to recommend a problem. So its a recommendation problem achieved through classification technique. Here for each order_id and product_id pair the model predicts 0 or 1. And then we create a dictionary and using order_id as key we create list of all products to be reorderd and set it as the value of dictionary for that specific key.


In [2]:
import pandas as pd
def predict_function(df, test_point):
    user_df = pd.DataFrame()
    user_df['max_number_of_orders'] = df.groupby(by='user_id')
    user_df['order_number'].aggregate('max').astype(np.uint8)
    user_df['mean_number_of_orders'] = df.groupby(by='user_id')['order_number'].aggregate('mean').astype(np.uint8)
    user_df['min_number_of_orders'] = df.groupby(by='user_id')
    user_df['order_number'].aggregate('min').astype(np.uint8)

    # Features Using Average Products
    max_order = pd.DataFrame()
    average_products= pd.DataFrame()
    max_order['total_products_per_order'] = df.groupby(by=['user_id', 'order_id'])
    max_order['product_id'].aggregate('count').astype(np.uint8)
    average_products['average_user_product'] = max_order.groupby(by=['user_id'])
    average_products['total_products_per_order'].aggregate('mean').astype(np.float16)

    # Features Using Day Of Week
    day_of_week = pd.DataFrame()
    day_of_week['max_order_day'] = df.groupby(by=['user_id'])['order_dow'].aggregate(lambda x : stats.mode(x)[0]).astype(np.uint8)
    day_of_week = day_of_week.reset_index()

    # Features Using Hour of Day
    hour_of_day = pd.DataFrame()
    hour_of_day['max_order_by_hour'] = df.groupby(by=['user_id'])['order_hour_of_day'].aggregate(lambda x : stats.mode(x)[0]).astype(np.uint8)
    hour_of_day = hour_of_day.reset_index()

    # Reorder Features
    reorder_by_user = pd.DataFrame()
    reorder_by_user['total_reorders'] = df[df['reordered']==1].groupby('user_id')['reordered'].aggregate('count').astype(np.uint8)
    reorder_by_user['total_non_reorders'] = df[df['reordered']==0].groupby('user_id')['reordered'].aggregate('count').astype(np.uint8)
    reorder_by_user['reorder_ratio'] =reorder_by_user['total_reorders']/(reorder_by_user['total_reorders'] +
    reorder_by_user['total_non_reorders']).astype(np.float16)

    # Merging All The Features
    user_df = pd.merge(user_df, average_products, on='user_id', how='left')
    user_df = pd.merge(user_df, day_of_week, on='user_id', how='left')
    user_df = pd.merge(user_df, hour_of_day, on='user_id', how='left')
    user_df = pd.merge(user_df, reorder_by_user, on='user_id', how='left')

    # Deleting the intermmediate dataframes and handling the NaN values if exists
    del average_products, day_of_week, hour_of_day, reorder_by_user, prior
    user_df = user_df.fillna(0)

    # Creating Product Dataframe
    product_df = pd.DataFrame()
    product_df['times_product_ordered'] = df.groupby(by='product_id')
    product_df['order_id'].aggregate('count').astype(np.uint16)

    # Reorder DataFrame Features
    reorder_products = pd.DataFrame()
    reorder_products['product_reorder_ratio'] = df.groupby('product_id')['reordered'].aggregate('mean').astype(np.float16)
    reorder_products['product_not_reorder_time'] = df[df['reordered']==0].groupby('product_id')['reordered'].aggregate('count').astype(np.uint8)

    # Add to cart features
    add_to_cart = pd.DataFrame()
    add_to_cart['average_product_cart_position'] = df.groupby(by='product_id')['add_to_cart_order'].aggregate('mean').astype(np.float16)
    add_to_cart['late_product_cart_position'] = df.groupby(by='product_id')['add_to_cart_order'].aggregate('max').astype(np.float16)
    add_to_cart['early_product_cart_position'] = df.groupby(by='product_id')['add_to_cart_order'].aggregate('min').astype(np.float16)

    # Day Of Week Features
    dow_product = pd.DataFrame()
    dow_product['average_product_order_day'] = df.groupby(by='product_id')['order_dow'].aggregate('mean').astype(np.float16)
    dow_product['late_product_order_day'] = df.groupby(by='product_id')['order_dow'].aggregate('max').astype(np.float16)
    dow_product['early_product_order_day'] = df.groupby(by='product_id')['order_dow'].aggregate('min').astype(np.float16)

    # Order Hour Of Day Features
    order_hour_of_day_product = pd.DataFrame()
    order_hour_of_day_product['average_product_hour'] = df.groupby(by='product_id')['order_hour_of_day'].aggregate('mean').astype(np.float16)
    order_hour_of_day_product['late_product_hour'] = df.groupby(by='product_id')['order_hour_of_day'].aggregate('max').astype(np.float16)
    order_hour_of_day_product['early_product_hour'] = df.groupby(by='product_id')['order_hour_of_day'].aggregate('min').astype(np.float16)

    # Merging all the dataframes
    product_df = pd.merge(product_df, reorder_products, on='product_id', how='left')
    product_df = pd.merge(product_df, add_to_cart, on='product_id', how='left')
    product_df = pd.merge(product_df, dow_product, on='product_id', how='left')
    product_df = pd.merge(product_df, order_hour_of_day_product, on='product_id', how='left')

    # User Product Dataframe
    user_product_df = pd.DataFrame()
    user_product_df['times_product_by_user'] = df.groupby(by=['user_id', 'product_id'])['order_id'].aggregate('count').astype(np.uint8)
    user_product_df = user_product_df.reset_index()

    # Products Bought DataFrame
    product_bought = pd.DataFrame()
    product_bought['times_bought'] = df.groupby(by=['user_id', 'product_id'])['order_id'].aggregate('count').astype(np.uint8)
    product_bought['earliest_order_position'] = df.groupby(by=['user_id', 'product_id'])['order_number'].aggregate('min').astype(np.uint8)
    product_bought = product_bought.reset_index()

    # Order By User DataFrame
    order_by_user = pd.DataFrame()
    order_by_user['total_orders_by_user'] = df.groupby('user_id')['order_number'].aggregate('max').astype(np.uint16)
    order_by_user = order_by_user.reset_index()

    merged_user_order = pd.merge(order_by_user, product_bought, how='right', on='user_id')
    merged_user_order['order_range'] = merged_user_order['total_orders_by_user'] - merged_user_order['earliest_order_position'] + 1

    user_product_merged_df = pd.merge(merged_user_order, user_product_df, on=['user_id', 'product_id'], how='left')
    user_product_merged_df['user_pro_reorder_ratio'] = user_product_merged_df['times_bought'] / user_product_merged_df['order_range']
    user_product_merged_df = user_product_merged_df.drop(['total_orders_by_user','order_range','times_product_by_user','earliest_order_position','times_bought'],axis=1)
    product_bought.drop(['earliest_order_position'],axis=1, inplace=True)
    product_bought = pd.merge(product_bought, user_product_merged_df, on=['user_id','product_id'], how='left')
    
    prior_order_df['order_number_back'] = (df.groupby(by=['user_id'])['order_number'].transform(max) - prior_order_df['order_number'] + 1).astype(np.uint8)
    temp = prior_order_df.loc[prior_order_df['order_number_back'] <= 10]

    recent_orders = pd.DataFrame()
    recent_orders['recent_ten'] = (temp.groupby(by=['user_id', 'product_id'])['order_id'].aggregate('count')).astype(np.uint16)
    recent_orders['recent_ten_ratio'] = (recent_orders['recent_ten'] / 10.0).astype(np.float16)

    product_bought = pd.merge(product_bought, recent_orders, on=['user_id', 'product_id'], how='left')
    product_bought = product_bought.fillna(0)
    del temp, recent_orders

    final_data = pd.merge(product_bought, user_df, on='user_id', how='left')
    final_data = pd.merge(final_data, product_df, on='product_id', how='left')
    del product_bought, product_df, user_df

    df_new = test_point['user_id', 'order_id']
    
    final_data_new = pd.merge(final_data, df, on='user_id', how='left')
    final_data_new = final_data_new.drop('reordered',axis=1)
    final_data_new = pd.merge(final_data_new, df_new, on='user_id', how='left')

    final_data_train = final_data_new.set_index(['user_id', 'product_id'])
    final_data_train.drop(['order_id'], axis=1, inplace=True)

    model = pickle.load(open('/content/drive/My Drive/Sequence_To_Sequence/tuned_lgb.pkl','rb'))
    
    final_data_train['reordered'] = (model.predict_proba(final_data_train)[:, 1] >=0.21).astype('int')

    final_test_data = final_data_train[['product_id', 'user_id', 'prediction']]
    
    user_product = dict()
    for row in tqdm(final_test_data.itertuples()):
        if row.prediction== 1:
            try:
                user_product[row.order_id] += ' ' + str(row.product_id)
            except:
                user_product[row.order_id] = str(row.product_id)

    for order in final_df.order_id:
        if order not in user_product:
            user_product[order] = 'None'

    return user_product.values()

# Second Function Description

1. The function takes a test_point as input. Now we look for the datapoint with same user_id, product_id, order_id.

2. If the test_point exists in the train data we get its label.

3. Now we load the model and predict on the test_point and calculate the f1_score with the true label.

4. If the test_point doesn't exists in the train data so we don't have the true label and we can't get the f1_score so we return the message <b>"Since the test point doesn't exits in train data so we don't have real label so we can't return metrics score.".</b>

In [6]:
from sklearn.metrics import f1_score

def evaluate_metrics(test_point):

    model = pickle.load(open('/content/drive/My Drive/Sequence_To_Sequence/tuned_lgb.pkl','rb'))
    
    order_id = test_point['order_id'].values
    product_id = test_point['product_id'].values
    user_id = test_point['user_id'].values

    train_data = pickle.load(open('/content/drive/My Drive/Instacart Analysis/final_data_trainn.pkl','rb'))
    true_label = train_data.loc[((train_data['order_id']==order_id) and (train_data['product_id']==product_id) and (train_data['user_id']==user_id)), ['reordered']]

    if(true_label!='None'):
        prediction = (model.predict_proba(final_data_train)[:, 1] >=0.21).astype('int')
        return f1_score(prediction, true_label)
    else:
        return "Since the test point doesn't exits in train data so we don't have real label so we can't return metrics score."